<a href="https://colab.research.google.com/github/ultra151/cv/blob/main/8_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz


In [ ]:
!tar -xf images.tar.gz
!tar -xf annotations.tar.gz

In [ ]:
import os

current_dir = os.getcwd()
print("Current working directory:", current_dir)

In [ ]:
!ls

In [ ]:
print(os.listdir("images"))

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
annotation_file = 'annotations/trimaps/Abyssinian_1.png'
annotation_image = Image.open(annotation_file)
annotation_array = np.array(annotation_image)

unique_values = np.unique(annotation_array)
print("Unique values in the annotation image:", unique_values)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(annotation_array, cmap='viridis')
plt.colorbar()
plt.show()

In [ ]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf)
#print(annotation_array)

In [ ]:
input_dir = "images/"
target_dir = "annotations/trimaps/"
img_size = (160, 160)
num_classes = 3
batch_size = 32

In [ ]:
input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".jpg")
    ]
)

target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

In [ ]:
input_img_paths[:3]

In [ ]:
print("입력 샘플의 수:", len(input_img_paths))
print("타겟 샘플의 수:", len(target_img_paths))

In [ ]:
from IPython.display import Image, display
from keras.utils import load_img
from PIL import ImageOps

display(Image(filename=input_img_paths[9]))

img = ImageOps.autocontrast(load_img(target_img_paths[9]))
display(img)

In [ ]:
import keras
import numpy as np
from tensorflow import data as tf_data
from tensorflow import image as tf_image
from tensorflow import io as tf_io


In [ ]:
def get_dataset(batch_size, img_size, input_img_paths, target_img_paths, max_dataset_len=None):

    def load_img_masks(input_img_path, target_img_path):
        input_img = tf_io.read_file(input_img_path)
        input_img = tf_io.decode_jpeg(input_img, channels=3)
        input_img = tf_image.resize(input_img, img_size)
        input_img = tf_image.convert_image_dtype(input_img, "float32")

        target_img = tf_io.read_file(target_img_path)
        target_img = tf_io.decode_png(target_img, channels=1)
        target_img = tf_image.resize(target_img, img_size, method="nearest")
        target_img = tf_image.convert_image_dtype(target_img, "uint8")

        target_img -= 1
        return input_img, target_img

    if max_dataset_len:
        input_img_paths = input_img_paths[:max_dataset_len]
        target_img_paths = target_img_paths[:max_dataset_len]

    dataset = tf_data.Dataset.from_tensor_slices((input_img_paths, target_img_paths))
    dataset = dataset.map(load_img_masks, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.batch(batch_size)


In [ ]:
import tensorflow as tf
import numpy as np

data = np.array([1,2,3,4,5])

dataset = tf.data.Dataset.from_tensor_slices(data)
print(type(dataset))

for element in dataset:
    print(element.numpy())

In [ ]:
from keras import layers

def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x

    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])
        previous_block_activation = x

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)

        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.UpSampling2D(2)(x)

        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])
        previous_block_activation = x

    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    model = keras.Model(inputs, outputs)
    return model

In [ ]:
model = get_model(img_size, num_classes)
model.summary()

In [ ]:
import tensorflow as tf

tf.keras.utils.plot_model(model, show_shapes=True, dpi=64)

In [ ]:
import random
val_samples = 1000
random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_img_paths)

train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]

val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

train_dataset = get_dataset(batch_size, img_size, train_input_img_paths, train_target_img_paths, max_dataset_len=1000)
valid_dataset = get_dataset(batch_size, img_size, val_input_img_paths, val_target_img_paths)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(1e-4), loss="sparse_categorical_crossentropy")
callbacks = [keras.callbacks.ModelCheckpoint("oxford_segmentation.keras", save_best_only=True)]

epochs = 50
model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=valid_dataset,
    callbacks=callbacks,
    verbose=2,
)

In [ ]:
val_dataset = get_dataset(batch_size, img_size, val_input_img_paths, val_target_img_paths)
val_preds = model.predict(val_dataset)

print(val_preds[0])

In [ ]:
print(val_preds.shape)

In [ ]:
def display_mask(i):
    mask = np.argmax(val_preds[i], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    img = ImageOps.autocontrast(keras.utils.array_to_img(mask))
    display(img)

In [ ]:
i = 10
display(Image(filename=val_input_img_paths[i]))

img = ImageOps.autocontrast(load_img(val_target_img_paths[i]))
display(img)

display_mask(i)

In [ ]:
predicted_classes = np.argmax(val_preds[10], axis=-1)
mask_positions = np.where(predicted_classes == 0)
print("경계 픽셀의 (행, 열) 포지션:", mask_positions)

In [ ]:
print(mask_positions[0][0])
print(mask_positions[1][0])